In [1]:
from keras.layers import Dense, LSTM
# from keras.layers import Input, Dropout, Activation
# from keras.layers import Convolution2D, MaxPooling2D, Reshape
from keras.layers import Convolution1D, MaxPooling1D, Flatten, Lambda, GRU
from keras.models import Sequential
from keras.layers.wrappers import TimeDistributed #输入至少为3D张量
# from matplotlib import pyplot as plt
# import random
import argparse
import pandas as pd
import time
from keras.preprocessing.sequence import TimeseriesGenerator
import numpy as np
from sklearn.model_selection import KFold
import keras.backend as K
from evaluation import calMetrix 
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
data = pd.read_csv('E:/LSTM_5_1/train_2018Q1_model_2.csv')

In [3]:
def DataLoader(data, numstep, length):

    serial_number = data['serial_number'].value_counts()

    input = []
    label = []

    for diskname in serial_number.index.sort_values():
        
        data_diskname = data[data.serial_number == diskname]


        value = data_diskname.iloc[:, length:-1].values
        max = np.max(value)
        min = np.min(value)
        scalar = max - min 
        data_norm = list(map(lambda x: x / scalar, value))

        data_label = data_diskname.loc[:, 'label'].values.tolist()

        for i in range(len(data_norm) - numstep - 1):
            input.append(np.array(data_norm[i: i + numstep]))
            if(data_label[i + numstep] == 1):
                label.append(1)
            elif(data_label[i + numstep] == 0):
                label.append(0)

    return np.array(input, dtype='float32'), np.array(label, dtype='int32')

In [4]:
x, y = DataLoader(data, 7, 6)

In [5]:
def read_data(X, y):
    seed = 15
    np.random.seed(seed)
    n_splits = 5
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=seed)

    print('KFold = %d :'%n_splits)
    for train_index, test_index in kfold.split(X, y):
        train_X, test_X = X[train_index], X[test_index]
        train_y, test_y = y[train_index], y[test_index]
        yield train_X, train_y, test_X, test_y

In [41]:
nb_epoch = 10
# number_of_batch = 100
batch_size = 10
# frame_row = 28
# frame_col = 2
# channels = 1
output_dim = 1

def classifyRes(arr):
    arr = arr.reshape(arr.size,)
    for i in range(arr.size):
        arr[i] = 1 if arr[i] > 0.5 else 0
    return arr

def model_fit(train_X, train_y, test_X, test_y):
    # define model
    model = Sequential()
    model.add(Lambda(lambda x: K.expand_dims(x, -1)))
    model.add(TimeDistributed(Convolution1D(1, 5, input_dim = (7, 13), border_mode = "same", W_regularizer = l2(0.01)), input_shape=train_X.shape[1:]))
    model.add(TimeDistributed(MaxPooling1D(10, border_mode = "same")))
    model.add(TimeDistributed(Convolution1D(1, 5, border_mode = "same", W_regularizer = l2(0.01))))
    model.add(TimeDistributed(MaxPooling1D(10, border_mode = "same")))
    model.add(Lambda(lambda x: K.squeeze(x, -1)))
    model.add(GRU(300, return_sequences = True, W_regularizer = l2(0.01), U_regularizer = l2(0.01)))
    model.add(TimeDistributed(Dense(1, activation='sigmoid')))
 
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.build(train_X.shape)
    model.summary()
    model.fit(train_X, train_y,batch_size=batch_size, nb_epoch=nb_epoch, verbose=0, validation_data=(test_X, test_y))

    test_y = test_y.reshape(test_y.size, 1)
    predict_y = model.predict(test_X)
    predict_y = predict_y.reshape(predict_y.size, 1)
    predict_y = classifyRes(predict_y)
    #calMetrix(__file__, predict_y, test_y)
    t0 = time.strftime('%Y%m%d%H%M%S', time.localtime(time.time()))

In [42]:
for train_X, train_y, test_X, test_y in read_data(x, y):
    model_fit(train_X, train_y, test_X, test_y)

KFold = 5 :
Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_32 (Lambda)           (17060, 7, 13, 1)         0         
_________________________________________________________________
time_distributed_80 (TimeDis (17060, 7, 13, 1)         6         
_________________________________________________________________
time_distributed_81 (TimeDis (17060, 7, 2, 1)          0         
_________________________________________________________________
time_distributed_82 (TimeDis (17060, 7, 2, 1)          6         
_________________________________________________________________
time_distributed_83 (TimeDis (17060, 7, 1, 1)          0         
_________________________________________________________________
lambda_33 (Lambda)           (17060, 7, 1)             0         
_________________________________________________________________
gru_13 (GRU)                 (17060, 7, 3

ValueError: Error when checking target: expected time_distributed_84 to have 3 dimensions, but got array with shape (17060, 1)

In [45]:
model = Sequential()
#model.add(TimeDistributed(cnn))
model.add(Lambda(lambda x: K.expand_dims(x, -1)))
model.add(TimeDistributed(Convolution1D(128, 4, border_mode='same'), input_shape=train_X.shape[1:]))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
#model.add(Lambda(lambda x: K.squeeze(x, -1)))
model.add(LSTM(128, return_sequences=True, name="lstm_layer0")) #output = batch * step * output_size
model.add(LSTM(128, return_sequences=False, name="lstm_layer1")) #output = batch * output_size(last step of every batch)
# model.add(LSTM(100, return_sequences=True, name="lstm_layer2"))
model.add(Dense(output_dim, activation='sigmoid'))
# model.add(GlobalAveragePooling1D(name="global_avg"))
 
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

model.build(x.shape)
model.summary()

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_37 (Lambda)           (21325, 7, 13, 1)         0         
_________________________________________________________________
time_distributed_91 (TimeDis (21325, 7, 13, 128)       640       
_________________________________________________________________
time_distributed_92 (TimeDis (21325, 7, 6, 128)        0         
_________________________________________________________________
time_distributed_93 (TimeDis (21325, 7, 768)           0         
_________________________________________________________________
lstm_layer0 (LSTM)           (21325, 7, 128)           459264    
_________________________________________________________________
lstm_layer1 (LSTM)           (21325, 128)              131584    
_________________________________________________________________
dense_30 (Dense)             (21325, 1)              